<a href="https://colab.research.google.com/github/lalityadav1980/ml_learnig/blob/main/ibm_new_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit qiskit-ibm-runtime qiskit-aer qiskit-machine-learning torch scikit-learn matplotlib pandas

In [ ]:
import os

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import Sampler

from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.connectors import TorchConnector

import pandas as pd
from qiskit_aer import Aer, AerSimulator
import matplotlib.pyplot as plt
import time
from scipy.stats import skew, kurtosis

start_time = time.time()
print("GPU Available:", torch.cuda.is_available())
# Add this line to define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")
API_TOKEN = "4ed9d81c24c1d6fb39f4f076976e930b0a0b57cf633303cafd3216068b4a268283b4a4bb0ca9d3ae6a787caf2b98fca3f4044a9051cb634ffb72f2eabdc6784b"
# === Authenticate and Connect to IBM Quantum Service ===
try:
    service = QiskitRuntimeService(
        channel="ibm_quantum",
        token=API_TOKEN
    )
    print("✅ Successfully authenticated with IBM Quantum.")

except Exception as e:
    print(f"❌ Authentication failed: {e}")


# ================== Create Parametric Circuit ==================
def create_quantum_circuit(num_qubits, num_features):
    # Define input and weight parameters
    input_params = ParameterVector('x', length=num_features)
    weight_params = ParameterVector('θ', length=num_qubits * 8)  # Adjusted for 3 layers

    qc = QuantumCircuit(num_qubits)

    # 1️⃣ Encode features
    for i in range(num_features):
        qc.rx(input_params[i], i % num_qubits)

    # 2️⃣ Initial Variational Layer
    for i in range(num_qubits):
        qc.ry(weight_params[i], i)
    for i in range(0, num_qubits, 2):
        if i + 1 < num_qubits:
            qc.cz(i, i + 1)
    for i in range(num_qubits):
        qc.rz(weight_params[num_qubits + i], i)

    qc.barrier()  # Add barrier for clarity

    # 3️⃣ Enhanced Variational Layers (Add More Layers for Higher Capacity)
    for layer in range(3):  # Repeat 3 times for deeper layers
        for i in range(num_qubits):
            ry_idx = (2 + layer * 2) * num_qubits + i  # Dynamic index calculation
            rz_idx = (3 + layer * 2) * num_qubits + i

            qc.ry(weight_params[ry_idx], i)
            qc.rz(weight_params[rz_idx], i)

        # Add entanglement (circular CZ connections)
        for i in range(num_qubits):
            qc.cz(i, (i + 1) % num_qubits)  # Circular entanglement

        qc.barrier()  # Optional: Improves circuit visualization

    # 4️⃣ Measurement
    qc.measure_all()

    return qc, input_params, weight_params


def process_last_records(historical_data, window_size=5):
    """
    Process the last N historical records to prepare input for prediction.
    """
    last_records = historical_data.tail(window_size)
    return last_records.values.flatten().reshape(1, -1)


# Modified predict_next_draw function
def predict_next_draw(model, scaler, historical_data):
    input_data = process_last_records(historical_data)
    scaled_data = scaler.transform(input_data)

    with torch.no_grad():
        # Move input to device and convert to float32
        input_tensor = torch.tensor(scaled_data, dtype=torch.float32).to(device)
        probs = model(input_tensor)

    return postprocess_predictions(probs.cpu().numpy()[0])


# ================== Build Samplers ==================
def build_local_simulator_sampler():
    """
    Create a local simulator-based Sampler using Qiskit Primitives.
    """
    simulator = AerSimulator(method='statevector')  # or 'qasm'
    print("Available Methods:", simulator.available_methods())
    local_sampler = Sampler()
    return local_sampler


def build_hardware_sampler(num_qubits):
    # Fetch available backends that are operational and meet qubit requirements
    possible_backends = [
        backend for backend in service.backends()
        if (not backend.configuration().simulator and
            backend.status().operational and
            backend.configuration().num_qubits >= num_qubits)
    ]

    if not possible_backends:
        raise ValueError("No operational real device found with enough qubits.")

    # Select the least busy backend
    least_busy_backend = min(possible_backends, key=lambda b: b.status().pending_jobs)
    print(f"🔗 Connecting to backend: {least_busy_backend.name}")

    # ✅ Set shots using `default_shots`
    hardware_sampler = SamplerV2(mode=least_busy_backend, options={"default_shots": 256})

    return hardware_sampler


# ================== Hybrid Model (Accepts Any Sampler) ==================
class HybridQuantumNet(nn.Module):
    def __init__(self, input_dim, num_qubits, sampler):
        super().__init__()

        # 1. Create parametric circuit
        qc, input_params, weight_params = create_quantum_circuit(num_qubits, input_dim)

        # Print circuit details for debugging
        print(qc.draw())
        print("Circuit depth:", qc.depth())
        print("Gate counts:", qc.count_ops())

        # 2. Build QNN
        self.quantum_nn = TorchConnector(
            SamplerQNN(
                circuit=qc,
                input_params=input_params,
                weight_params=weight_params,
                sampler=sampler,  # local or hardware
                input_gradients=True
            )
        )

        # 3. Classical network
        self.classical_fc = nn.Sequential(
            nn.Linear(2 ** num_qubits, 256),  # Match quantum output
            nn.LeakyReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 49),
            nn.Sigmoid()  # For multi-label classification
        )

    def forward(self, x):
        x = self.quantum_nn(x)
        return self.classical_fc(x)


def create_sequences(data, window_size=5):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i - window_size:i].flatten())
        # Create multi-hot encoding for 7 numbers
        target = np.zeros(49)
        for num in data[i][:7]:
            target[num - 1] = 1  # Numbers 1-49 to indices 0-48
        y.append(target)
    return np.array(X), np.array(y)


##############################################################################
# Utility Functions
##############################################################################
def compute_rolling_frequency_features(df: pd.DataFrame, window: int = 5, max_ball: int = 49) -> pd.DataFrame:
    """
    Computes rolling frequency features for each lottery number.

    Parameters:
    - df (pd.DataFrame): Input DataFrame containing lottery numbers.
    - window (int): Number of past draws to consider.
    - max_ball (int): Maximum lottery number.

    Returns:
    - pd.DataFrame: DataFrame with added frequency features.
    """
    df = df.copy().reset_index(drop=True)
    numeric_cols = [c for c in df.columns if "winning_number" in c or "additional_number" in c]

    for b in range(1, max_ball + 1):
        df[f"freq_{b}"] = 0

    for i in range(len(df)):
        start = max(0, i - window)
        end = i  # Exclusive
        recent_draws = df.iloc[start:end][numeric_cols]
        counts = {k: 0 for k in range(1, max_ball + 1)}

        for _, row in recent_draws.iterrows():
            for val in row.values:
                if val in counts:
                    counts[val] += 1

        for k in range(1, max_ball + 1):
            df.at[i, f"freq_{k}"] = counts[k]

    return df


def compute_sequence_features_with_numbers(df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes features related to sequences and gaps in the lottery numbers,
    including the actual numbers in the sequences and their frequencies,
    converted to Python integers.

    Parameters:
    - df (pd.DataFrame): Input DataFrame containing lottery numbers.

    Returns:
    - pd.DataFrame: DataFrame with added sequence and gap features.
    """
    df = df.copy()
    numeric_cols = [c for c in df.columns if "winning_number" in c or "additional_number" in c]

    one_gap_numbers = []
    all_one_gap_pairs = []  # To collect all unique one-gap pairs

    for _, row in df[numeric_cols].iterrows():
        numbers = sorted(row.values)  # Sort numbers for easier comparison
        one_gap_nums = []

        for i in range(len(numbers) - 1):
            # One-gap numbers
            if numbers[i + 1] - numbers[i] == 2:
                pair = (int(numbers[i]), int(numbers[i + 1]))
                one_gap_nums.append(pair)
                all_one_gap_pairs.append(pair)

        one_gap_numbers.append(one_gap_nums)

    # Add counts and numbers as features
    df["one_gap_numbers"] = one_gap_numbers

    # Identify unique one-gap pairs
    unique_one_gap_pairs = set(all_one_gap_pairs)

    # Calculate frequency and presence of each pair
    for pair in unique_one_gap_pairs:
        col_freq = f"freq_{pair[0]}_{pair[1]}"
        col_has = f"has_{pair[0]}_{pair[1]}"

        # Frequency of the pair in the entire dataset
        freq = sum(pair in gaps for gaps in one_gap_numbers)

        # Add columns for frequency and presence
        df[col_freq] = df["one_gap_numbers"].apply(lambda x: x.count(pair))
        df[col_has] = df["one_gap_numbers"].apply(lambda x: 1 if pair in x else 0)
    df.drop(columns=["one_gap_numbers"], inplace=True)
    print("Added sequence features with frequencies and presence indicators.")
    return df


def compute_longer_consecutive_sequences(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    numeric_cols = [c for c in df.columns if "winning_number" in c or "additional_number" in c]

    max_sequence_length = []
    consecutive_sequences = []
    sequence_counter = {}

    for _, row in df[numeric_cols].iterrows():
        numbers = sorted(row.values)
        current_sequence = []
        all_sequences = []
        max_length = 0

        for i in range(len(numbers)):
            if i == 0 or numbers[i] - numbers[i - 1] == 1:
                current_sequence.append(int(numbers[i]))
            else:
                if len(current_sequence) > 1:
                    all_sequences.append(current_sequence)
                    max_length = max(max_length, len(current_sequence))
                current_sequence = [int(numbers[i])]

        if len(current_sequence) > 1:
            all_sequences.append(current_sequence)
            max_length = max(max_length, len(current_sequence))

        max_sequence_length.append(max_length)
        consecutive_sequences.append(all_sequences)

        # Count sequences
        for seq in all_sequences:
            key = '_'.join(map(str, seq))
            sequence_counter[key] = sequence_counter.get(key, 0) + 1

    # Add the sequences column temporarily
    df["longer_consecutive_sequences"] = consecutive_sequences

    # Collect all new features in a dictionary
    new_features = {}
    for seq in sequence_counter:
        freq_col = f"freq_{seq}"
        has_col = f"has_{seq}"

        freq_data = df["longer_consecutive_sequences"].apply(
            lambda x: sum(1 for s in x if '_'.join(map(str, s)) == seq)
        )
        has_data = freq_data.apply(lambda x: 1 if x > 0 else 0)

        new_features[freq_col] = freq_data
        new_features[has_col] = has_data

    # Concatenate all new features at once
    new_features_df = pd.DataFrame(new_features)
    df = pd.concat([df, new_features_df], axis=1)

    # Drop the temporary column
    df.drop(columns=["longer_consecutive_sequences"], inplace=True)

    # Defragment the DataFrame
    df = df.copy()

    print("Optimized: Added features for longer consecutive sequences without performance warnings.")
    return df


def compute_even_odd_ratio(df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes the ratio of even numbers in each draw.

    Parameters:
    - df (pd.DataFrame): Input DataFrame containing lottery numbers.

    Returns:
    - pd.DataFrame: DataFrame with added even-odd ratio feature.
    """
    df = df.copy()
    numeric_cols = [c for c in df.columns if "winning_number" in c or "additional_number" in c]

    ratios = []
    for _, row in df[numeric_cols].iterrows():
        even_count = sum(n % 2 == 0 for n in row.values)
        ratio = even_count / len(row.values)  # Out of 7
        ratios.append(ratio)

    df["even_odd_ratio"] = ratios
    return df


def compute_hot_cold_numbers(df: pd.DataFrame, window: int = 10) -> pd.DataFrame:
    """
    Computes hot and cold numbers based on their frequency in recent draws.

    Parameters:
    - df (pd.DataFrame): Input DataFrame containing lottery numbers.
    - window (int): Number of past draws to consider.

    Returns:
    - pd.DataFrame: DataFrame with added hot and cold number features as separate columns.
    """
    df = df.copy()
    numeric_cols = [c for c in df.columns if "winning_number" in c or "additional_number" in c]

    for i in range(len(df)):
        start = max(0, i - window)
        end = i  # Exclusive
        recent_draws = df.iloc[start:end][numeric_cols]
        counts = recent_draws.stack().value_counts().to_dict()

        # Hot numbers: most frequent in recent draws
        hot_numbers = sorted(counts, key=counts.get, reverse=True)[:6]
        # Cold numbers: least frequent in recent draws
        cold_numbers = sorted(counts, key=counts.get)[:6]

        # Add hot and cold numbers as separate columns
        for j, num in enumerate(hot_numbers):
            df.at[i, f"hot_number_{j + 1}"] = num
        for j, num in enumerate(cold_numbers):
            df.at[i, f"cold_number_{j + 1}"] = num

    return df


def compute_gaps_between_occurrences(df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes the number of draws since each number last appeared.

    Parameters:
    - df (pd.DataFrame): Input DataFrame containing lottery numbers.

    Returns:
    - pd.DataFrame: DataFrame with added gap features.
    """
    df = df.copy()
    numeric_cols = [c for c in df.columns if "winning_number" in c or "additional_number" in c]

    last_seen = {num: -1 for num in range(1, 50)}
    gaps = []

    for i in range(len(df)):
        row = df.iloc[i][numeric_cols]
        current_gaps = []
        for num in range(1, 50):
            if num in row.values:
                current_gaps.append(0)
                last_seen[num] = i
            else:
                current_gaps.append(i - last_seen[num] if last_seen[num] != -1 else 0)
        gaps.append(current_gaps)

    gap_cols = [f"gap_{num}" for num in range(1, 50)]
    df[gap_cols] = gaps
    return df


def compute_number_pairs_triplets(df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes frequently occurring pairs and triplets of numbers.

    Parameters:
    - df (pd.DataFrame): Input DataFrame containing lottery numbers.

    Returns:
    - pd.DataFrame: DataFrame with added pair and triplet features.
    """
    df = df.copy()
    numeric_cols = [c for c in df.columns if "winning_number" in c or "additional_number" in c]

    # Count pairs and triplets
    pairs = {}
    triplets = {}

    for i in range(len(df)):
        row = df.iloc[i][numeric_cols].values
        # Pairs
        for j in range(len(row)):
            for k in range(j + 1, len(row)):
                pair = tuple(sorted((row[j], row[k])))
                pairs[pair] = pairs.get(pair, 0) + 1
        # Triplets
        for j in range(len(row)):
            for k in range(j + 1, len(row)):
                for l in range(k + 1, len(row)):
                    triplet = tuple(sorted((row[j], row[k], row[l])))
                    triplets[triplet] = triplets.get(triplet, 0) + 1

    # Add top pairs and triplets as features
    top_pairs = sorted(pairs, key=pairs.get, reverse=True)[:5]
    top_triplets = sorted(triplets, key=triplets.get, reverse=True)[:5]

    for i, pair in enumerate(top_pairs):
        df[f"top_pair_{i + 1}"] = ",".join(map(str, pair))
    for i, triplet in enumerate(top_triplets):
        df[f"top_triplet_{i + 1}"] = ",".join(map(str, triplet))

    return df


def compute_statistical_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes statistical features (mean, std, skewness, kurtosis) for each draw.

    Parameters:
    - df (pd.DataFrame): Input DataFrame containing lottery numbers.

    Returns:
    - pd.DataFrame: DataFrame with added statistical features.
    """
    df = df.copy()
    numeric_cols = [c for c in df.columns if "winning_number" in c or "additional_number" in c]

    df["mean"] = df[numeric_cols].mean(axis=1)
    df["std"] = df[numeric_cols].std(axis=1)
    df["skewness"] = df[numeric_cols].apply(lambda x: skew(x), axis=1)
    df["kurtosis"] = df[numeric_cols].apply(lambda x: kurtosis(x), axis=1)

    return df


def get_feature_columns(df: pd.DataFrame) -> list:
    """
    Identifies feature columns by excluding specific columns related to basic lottery information.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.

    Returns:
    - List of feature columns.
    """
    # ✅ Columns to exclude
    exclude_cols = [
        "Date_numeric",
        "winning_number_1", "winning_number_2", "winning_number_3",
        "winning_number_4", "winning_number_5", "winning_number_6",
        "additional_number"
    ]

    # 🚀 Automatically detect all other feature columns
    feature_cols = [col for col in df.columns if col not in exclude_cols]

    print(f"Identified {len(feature_cols)} feature columns.")
    return feature_cols


# ================== Data Processing ==================
def preprocess_data(data_path, window_size=5):
    """
    Preprocess the data and split it into training, validation, and test sets.
    Computes additional statistical features and saves the splits into separate CSV files.
    """
    # Load and sort data by date
    df = pd.read_csv(data_path).sort_values('Date', ascending=True).reset_index(drop=True)
    df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%y", errors="coerce")
    df["Date_numeric"] = df["Date"].astype("int64") // 1e9
    df.drop(columns=["Date"], inplace=True)

    # Apply statistical functions to the DataFrame
    #df = compute_rolling_frequency_features(df, window=window_size)
    df = compute_sequence_features_with_numbers(df)
    df = compute_longer_consecutive_sequences(df)

    df = compute_even_odd_ratio(df)
    df = compute_hot_cold_numbers(df, window=window_size)
    df = compute_statistical_features(df)

    # ✅ Remove rows with any NaN or empty values
    df = df.dropna()  # Remove rows with NaN values
    df = df[~(df == '').any(axis=1)]  # Remove rows with empty string values

    print(f"Cleaned Data: Removed {len(pd.read_csv(data_path)) - len(df)} rows with empty or NaN values.")


    pd.DataFrame(df).to_csv("/content/sample_data/fetaure_processed.csv", index=False)
    # Calculate split indices
    total_samples = len(df)
    train_size = int(total_samples * 0.8)  # 80% for training
    val_size = int(total_samples * 0.1)  # 10% for validation
    test_size = total_samples - train_size - val_size  # Remaining 10% for testing

    # Split data into training, validation, and test sets
    train_df = df.iloc[:train_size]
    val_df = df.iloc[train_size:train_size + val_size]
    test_df = df.iloc[train_size + val_size:]

    # Save splits to separate CSV files
    train_df.to_csv("/content/sample_data/training_data.csv", index=False)
    val_df.to_csv("/content/sample_data/validation_data.csv", index=False)
    test_df.to_csv("/content/sample_data/test_data.csv", index=False)

    # Select new statistical feature columns
    feature_cols = get_feature_columns(df)

    # Process features for each split
    def process_split(split_df, is_train=False, train_df=None):
        if is_train:
            numbers = split_df[[f"winning_number_{i}" for i in range(1, 7)] + ["additional_number"]].values
            stats = split_df[feature_cols].values
        else:
            combined_df = pd.concat([train_df, split_df], axis=0).reset_index(drop=True)
            numbers = combined_df[[f"winning_number_{i}" for i in range(1, 7)] + ["additional_number"]].values
            stats = combined_df[feature_cols].values

        # Create sequences and features
        X_seq, y_seq = create_sequences(numbers, window_size)
        X_stats = stats[window_size:]

        return np.hstack([X_seq, X_stats]), y_seq

    # Process training data
    X_train, y_train = process_split(train_df, is_train=True)

    # Process validation data
    X_val, y_val = process_split(val_df, train_df=train_df)

    # Process test data
    X_test, y_test = process_split(test_df, train_df=pd.concat([train_df, val_df], axis=0))
    pd.DataFrame(X_test).to_csv("/content/sample_data/X_test_processed.csv", index=False)
    pd.DataFrame(y_test).to_csv("/content/sample_data/y_test_processed.csv", index=False)

    # Fit scaler on training data only
    scaler = MinMaxScaler(feature_range=(0, np.pi))
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


# ================== Training with Model Checkpoint ==================
def train_model(model, X_train, y_train, X_val, y_val, epochs=10, checkpoint_path='best_model.pth'):
    # Move model to the appropriate device
    model = model.to(device)
    print(f"Training on device: {device}")

    BATCH_SIZE = 64

    # Load pre-trained model if available
    if os.path.exists(checkpoint_path):
        model.load_state_dict(torch.load(checkpoint_path, map_location=device))
        print(f"✅ Loaded pre-trained model from {checkpoint_path}.")
        return model

    # Define loss function, optimizer, and scheduler
    criterion = nn.BCEWithLogitsLoss()  # For multi-label classification
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

    # Convert input data to tensors and move to device
    X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_t = torch.tensor(y_train, dtype=torch.float32).to(device)  # Targets must be float for BCEWithLogitsLoss
    X_val_t = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_t = torch.tensor(y_val, dtype=torch.float32).to(device)

    train_losses, val_losses = [], []
    best_val_loss = float('inf')
    patience, counter = 5, 0
    min_delta = 0.001  # Minimum improvement threshold
    overfit_threshold = 0.1  # Maximum allowed gap between training and validation loss

    # Training loop
    for epoch in range(epochs):
        model.train()
        epoch_train_loss = 0.0

        # Mini-batch training
        for i in range(0, len(X_train_t), BATCH_SIZE):
            # Get batch
            batch_X = X_train_t[i:i + BATCH_SIZE]
            batch_y = y_train_t[i:i + BATCH_SIZE]

            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)  # Use batch_y directly (no argmax)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_train_loss += loss.item()

        # Calculate average training loss for the epoch
        avg_train_loss = epoch_train_loss / (len(X_train_t) // BATCH_SIZE + 1)
        train_losses.append(avg_train_loss)

        # Validation phase
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_t)
            val_loss = criterion(val_outputs, y_val_t)  # Use full validation outputs
            val_losses.append(val_loss.item())

        # Print epoch statistics
        print(
            f"📈 Epoch [{epoch + 1}/{epochs}] - Training Loss: {avg_train_loss:.6f}, Validation Loss: {val_loss.item():.6f}")

        # Update learning rate scheduler
        scheduler.step(val_loss)

        # Check for early stopping conditions
        if val_loss.item() < best_val_loss - min_delta:
            best_val_loss = val_loss.item()
            torch.save(model.state_dict(), checkpoint_path)
            print(f"💾 Best model saved with validation loss: {best_val_loss:.6f}")
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("⏹️ Early stopping triggered due to lack of improvement.")
                break

        # Check for overfitting
        if avg_train_loss - val_loss.item() > overfit_threshold:
            print("⏹️ Overfitting detected. Stopping training.")
            break

    print("✅ Training completed successfully.")
    print(f"⏱️ Training completed in {time.time() - start_time:.2f} seconds")
    plot_loss(train_losses, val_losses)
    return model


# ================== Plotting Losses ==================
def plot_loss(train_losses, val_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()  # Use show() instead of savefig()


# ================== Cache Quantum Model ==================
def cache_quantum_model(model, cache_path='quantum_model_cache.pth'):
    torch.save(model.state_dict(), cache_path)
    print(f"Quantum hardware model cached at {cache_path}.")


def load_cached_model(model, cache_path='quantum_model_cache.pth'):
    if os.path.exists(cache_path):
        model.load_state_dict(torch.load(cache_path))
        print(f"Loaded cached quantum model from {cache_path}")
    else:
        print(f"No cached model found at {cache_path}.")
    return model


# ================== Prediction vs Actual Plot ==================
def plot_predictions(predictions, actuals):
    plt.figure(figsize=(10, 6))
    plt.plot(predictions, label='Predicted', marker='o')
    plt.plot(actuals, label='Actual', marker='x')
    plt.xlabel('Sample Index')
    plt.ylabel('Values')
    plt.title('Predictions vs Actuals')
    plt.legend()
    plt.grid(True)
    plt.show()


# ================== Postprocessing ==================
def postprocess_predictions(probs, top_k=7):
    """Select numbers based on model confidence"""
    numbers = np.arange(1, 50)
    sorted_idx = np.argsort(-probs)  # Descending order
    selected = []

    for idx in sorted_idx:
        num = numbers[idx]
        if num not in selected:
            selected.append(num)
        if len(selected) == top_k:
            break

    return sorted(selected)


# ================== Main Execution ==================
def main():
    DATA_PATH = "/content/sample_data/ToTo.csv"
    NUM_QUBITS = 4

    # 1️⃣ Load data
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = preprocess_data(DATA_PATH)

    # 2️⃣ Train with Simulator
    local_sampler = build_local_simulator_sampler()
    model_sim = HybridQuantumNet(
        input_dim=X_train.shape[1],
        num_qubits=NUM_QUBITS,
        sampler=local_sampler
    )
    model_sim = train_model(model_sim, X_train, y_train, X_test, y_test, epochs=10)

    # Modified evaluation section in main()
    # 3️⃣ Evaluate Predictions (Simulator)
    with torch.no_grad():
        test_input = torch.tensor(X_test, dtype=torch.float32).to(device)
        predictions = model_sim(test_input)[:, :7].cpu().numpy()

    plot_predictions(predictions.flatten(), y_test.flatten())

    # 4️⃣ Quantum Hardware Model (Cache if available)
    hardware_sampler = build_hardware_sampler(NUM_QUBITS)
    model_hw = HybridQuantumNet(
        input_dim=X_train.shape[1],
        num_qubits=NUM_QUBITS,
        sampler=hardware_sampler
    )

    # Load cached model if exists
    model_hw = load_cached_model(model_hw)

    # If no cached model, train and save cache
    if not os.path.exists("/content/drive/MyDrive/quantum_model_cache.pth"):
        model_hw.load_state_dict(model_sim.state_dict())  # Transfer learning
        model_hw = train_model(model_hw, X_train, y_train, X_test, y_test, epochs=3)
        cache_quantum_model(model_hw)

    # 5️⃣ Final Predictions (Hardware)
    with torch.no_grad():
        hardware_predictions = model_sim(torch.tensor(X_test, dtype=torch.float32).to(device))[:, :7].cpu().numpy()

    plot_predictions(hardware_predictions.flatten(), y_test.flatten())

    # 6️⃣ 🔮 Predict the Next Draw Using Latest Historical Data
    historical_data = pd.read_csv(DATA_PATH)
    scaler = MinMaxScaler(feature_range=(0, np.pi))
    scaler.fit(X_train)  # Fit the scaler with training data for consistency

    next_draw_prediction = predict_next_draw(model_hw, scaler, historical_data)
    print(f"🎯 Predicted Next Draw Numbers: {next_draw_prediction}")


main()


GPU Available: True
Using device: cuda
✅ Successfully authenticated with IBM Quantum.
Added sequence features with frequencies and presence indicators.
Optimized: Added features for longer consecutive sequences without performance warnings.
Cleaned Data: Removed 1 rows with empty or NaN values.
Identified 336 feature columns.
Available Methods: ('automatic', 'statevector', 'density_matrix', 'stabilizer', 'matrix_product_state', 'extended_stabilizer', 'unitary', 'superop')


<ipython-input-2-b9eee394d412>:115: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  local_sampler = Sampler()


        ┌──────────┐┌──────────┐ ┌──────────┐┌───────────┐┌───────────┐»
   q_0: ┤ Rx(x[0]) ├┤ Rx(x[4]) ├─┤ Rx(x[8]) ├┤ Rx(x[12]) ├┤ Rx(x[16]) ├»
        ├──────────┤├──────────┤ ├──────────┤├───────────┤├───────────┤»
   q_1: ┤ Rx(x[1]) ├┤ Rx(x[5]) ├─┤ Rx(x[9]) ├┤ Rx(x[13]) ├┤ Rx(x[17]) ├»
        ├──────────┤├──────────┤┌┴──────────┤├───────────┤├───────────┤»
   q_2: ┤ Rx(x[2]) ├┤ Rx(x[6]) ├┤ Rx(x[10]) ├┤ Rx(x[14]) ├┤ Rx(x[18]) ├»
        ├──────────┤├──────────┤├───────────┤├───────────┤├───────────┤»
   q_3: ┤ Rx(x[3]) ├┤ Rx(x[7]) ├┤ Rx(x[11]) ├┤ Rx(x[15]) ├┤ Rx(x[19]) ├»
        └──────────┘└──────────┘└───────────┘└───────────┘└───────────┘»
meas: 4/═══════════════════════════════════════════════════════════════»
                                                                       »
«        ┌───────────┐┌───────────┐┌───────────┐┌───────────┐┌───────────┐»
«   q_0: ┤ Rx(x[20]) ├┤ Rx(x[24]) ├┤ Rx(x[28]) ├┤ Rx(x[32]) ├┤ Rx(x[36]) ├»
«        ├───────────┤├───────────┤├─────────

<ipython-input-2-b9eee394d412>:156: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  SamplerQNN(
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
